<a href="https://colab.research.google.com/github/dinesh-umkc/kdm/blob/main/ICP6_LCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LCA- Topic Modeling
#Objective
1. Get books from https://www.gutenberg.org 
2. Clean content from book data
3. Get topics using LCA
4. Get content from Reddit
5. Clean content from thread data
6. Get topics using LCA

In [7]:
import urllib
from bs4 import BeautifulSoup

from urllib.request import urlopen
from bs4 import BeautifulSoup
documents = []

import re
regex = re.compile(r'<[^>]+>')

def remove_html(string):
    return regex.sub('', string)

def getContent(url): 
  #url = "https://www.gutenberg.org/files/55/55-h/55-h.htm" 
  html = urlopen(url).read()
  soup = BeautifulSoup(html, features="html.parser")
  for script in soup(["script", "style"]):
    script.extract()
  text = soup.get_text()
  documents.append(remove_html(text))

url="https://www.gutenberg.org/files/55/55-h/55-h.htm"
getContent(url)
(documents[0])[:500]
url = "https://www.gutenberg.org/files/54/54-h/54-h.htm" 
getContent(url)
url = "https://www.gutenberg.org/files/33361/33361-h/33361-h.htm" 
getContent(url)
url = "https://www.gutenberg.org/files/22566/22566-h/22566-h.htm" 
getContent(url)
url = "https://www.gutenberg.org/files/26624/26624-h/26624-h.htm" 
getContent(url)



In [8]:
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

tfidf_vect = CountVectorizer(analyzer='word', stop_words = 'english')
df=tfidf_vect.fit_transform(documents)
vocab = tfidf_vect.get_feature_names_out()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
vocab

array(['00', '000', '10', ..., 'zoroaster', 'zuz', 'zy'], dtype=object)

In [ ]:
print(df.shape)

(5, 8851)


In [9]:
print(df[0])

  (0, 5966)	89
  (0, 3695)	99
  (0, 2645)	14
  (0, 8716)	28
  (0, 8697)	44
  (0, 5411)	169
  (0, 3303)	5
  (0, 860)	5
  (0, 8323)	28
  (0, 8255)	15
  (0, 7421)	19
  (0, 5481)	2
  (0, 8749)	19
  (0, 1915)	4
  (0, 6420)	2
  (0, 8573)	2
  (0, 1886)	12
  (0, 766)	71
  (0, 7827)	22
  (0, 4567)	18
  (0, 4108)	3
  (0, 5320)	4
  (0, 8794)	10
  (0, 5353)	10
  (0, 4644)	7
  :	:
  (0, 5427)	1
  (0, 2484)	1
  (0, 4910)	1
  (0, 401)	1
  (0, 8527)	1
  (0, 2005)	1
  (0, 1352)	1
  (0, 5957)	1
  (0, 4913)	1
  (0, 3775)	1
  (0, 5361)	1
  (0, 6889)	1
  (0, 5953)	1
  (0, 4675)	1
  (0, 5168)	1
  (0, 8434)	1
  (0, 1783)	1
  (0, 5136)	1
  (0, 2655)	1
  (0, 4749)	1
  (0, 5626)	1
  (0, 2958)	1
  (0, 5952)	1
  (0, 7601)	1
  (0, 5177)	1


In [ ]:
print (vocab[5177])

newsletter


In [10]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components = 5, doc_topic_prior=1)
lda.fit(df)
lda.components_[0].shape

(8851,)

In [11]:
import numpy as np 
topic_words = {}
n_top_words = 20
for topic, comp in enumerate(lda.components_):
    # print(topic, comp)
    word_idx = np.argsort(comp)[::-1][:n_top_words] #argsort to get index, and [::-1] to sort in descending
    # store the words most relevant to the topic
    topic_words[topic] = [vocab[i] for i in word_idx]
    # break
    
for topic, words in topic_words.items():
    print('Topic: %d' % topic)
    print('  %s' % ', '.join(words))

Topic: 0
  1993, tinkle, beaten, utf, texts, straighten, luckily, stock, greensward, cleared, snatched, melting, brittle, moss, raising, dizzy, deceived, messenger, glowed, housework
Topic: 1
  dorothy, said, little, man, pg, oz, asked, scarecrow, gutenberg, king, girl, wizard, project, ozma, tin, good, woodman, great, shaggy, head
Topic: 2
  1993, tinkle, beaten, utf, texts, straighten, luckily, stock, greensward, cleared, snatched, melting, brittle, moss, raising, dizzy, deceived, messenger, glowed, housework
Topic: 3
  said, tip, scarecrow, horse, jack, saw, woodman, tin, pumpkinhead, mombi, city, boy, bug, woggle, head, gutenberg, glinda, good, old, project
Topic: 4
  1993, tinkle, beaten, utf, texts, straighten, luckily, stock, greensward, cleared, snatched, melting, brittle, moss, raising, dizzy, deceived, messenger, glowed, housework


#Reddit Data

In [12]:
# !pip install praw
!pip install asyncpraw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# import praw
import asyncpraw as praw
from asyncpraw.models import MoreComments
reddit = praw.Reddit(
    user_agent="Comment Extraction (by u/Bitter_Researcher859)",
    client_id="5YUVEJjNjYgNfwrR0Jle_Q",
    client_secret="y1Yw5hcMa874BIzhbG5DSFnE9f_ncg",
    redirect_uri='http://127.0.0.1:65010/'
                                 'authorize_callback'
    # username="USERNAME",
    # password="PASSWORD"
)

In [13]:
import time
from tqdm.notebook import tqdm
topic_library = {
    'CryptoCurrency' : [ 'https://www.reddit.com/r/dogecoin/comments/lb3ant/whos_still_holding_doge/', 
                  'https://www.reddit.com/r/Superstonk/comments/na3akt/all_the_confirmation_bias_i_need_right_here_in/',
                  'https://www.reddit.com/r/CryptoCurrency/comments/n7rl2y/you_hear_about_the_kid_who_put_in_500_into_a/'],
    'Gaming': [
        'https://www.reddit.com/r/gaming/comments/lvcakc/boy_gamer/',
        'https://www.reddit.com/r/gaming/comments/kwi9yr/ea_will_no_longer_have_exclusive_rights_of_the/',
        'https://www.reddit.com/r/leagueoflegends/comments/q58hok/worlds_2021_prediction_tournament/',
    ],
    'AnimalsOnReddit': [
        'https://www.reddit.com/rpan/r/AnimalsOnReddit/m52wd4',
        'https://www.reddit.com/rpan/r/AnimalsOnReddit/mo59jj', 
        ]
}
for topic, urls in tqdm(topic_library.items()):
  for url in tqdm(urls):
    submission = await reddit.submission(url=url)
    submission.comments.replace_more(limit=0) # flatten tree
    all_comments = []
    comments = submission.comments # all comments
    # comments = submission.comments.list() # all comments
    for top_level_comment in submission.comments:
      if isinstance(top_level_comment, MoreComments):
          continue
      all_comments.append(top_level_comment.body)
      with open('{}-reddit_author.txt'.format(topic), 'a') as f:
          f.write(submission.selftext + '\n')
      with open('{}-reddit_comments.txt'.format(topic), 'a') as f:
        for i, comment in enumerate(comments):
          f.write(comment.body + '\n')
    # time.sleep(1.5)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: coroutine 'CommentForest.replace_more' was never awaited


AttributeError: ignored